### Get article

In [24]:
import os
import requests
from typing import List, Tuple
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from newsapi import NewsApiClient
from openai import OpenAI

load_dotenv(verbose=True)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

NEWS_TOKEN = os.getenv("NEWS_API_KEY")
newsapi = NewsApiClient(api_key=NEWS_TOKEN)
top_headlines = newsapi.get_top_headlines(sources='bbc-news', page_size=1)

In [25]:
top_headlines

{'status': 'ok',
 'totalResults': 10,
 'articles': [{'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'BBC News',
   'title': 'Ten people killed in New Orleans as vehicle ploughs into crowd',
   'description': 'Police say 35 others injured on Bourbon Street by attacker "hell-bent on carnage".',
   'url': 'https://www.bbc.co.uk/news/articles/crl378x8nnjo',
   'urlToImage': 'https://ichef.bbci.co.uk/ace/branded_news/1200/cpsprodpb/c3f4/live/96275010-c837-11ef-8f44-0922f468356e.png',
   'publishedAt': '2025-01-01T13:37:24.800693Z',
   'content': 'A CBS reporter saw multiple people with injuries on the ground at the intersection of Bourbon and Canal streets.\r\nA video confirmed as genuine by BBC Verify show a person lying on the ground and a cr… [+2053 chars]'}]}

In [26]:
def get_article_from_url(url: str) -> str:
    """
    입력받은 URL을 토대로 기사의 본문 태그를 찾아 본문 내용을 반환합니다.
    """
    # get HTML content of url
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # select p tag to get content
    paragraphs = soup.find_all('p')
    article_text = '\n'.join([para.get_text() for para in paragraphs])
    
    return article_text

In [27]:
context = get_article_from_url(top_headlines["articles"][0]["url"])
context

https://www.bbc.co.uk/news/articles/crl378x8nnjo


'The suspect in the New Orleans attack that killed 15 people on New Year\'s Day did not act alone, US investigators believe.\nShamsud-Din Jabbar, a 42-year-old US citizen, is believed to have driven a pick-up truck into a crowd on a busy New Orleans street, before exiting the vehicle and firing a weapon. He was shot dead by police at the scene.\nThe FBI says an Islamic State (IS) group flag was found inside the vehicle he was driving, while two improvised explosive devices were found nearby.\nFBI assistant special agent Alethea Duncan said the agency did not believe Jabbar was "solely responsible" and were investigating the incident as an "act of terrorism".\nMore on the New Orleans attack\nLIVE: Several people involved in IS-inspired attack\nThe victims: Aspiring nurse and ex-college football player among the dead\nThe attacker: What we know about Shamsud-Din Jabbar\nWatch: How day of deadly attack unfolded\nPresident Joe Biden said investigators were looking into whether the incident

### Create article

In [28]:
def convert_txt_to_steps(context: str, level: str):
    """
    입력받은 기사 내용(context)을 입력받은 수준(level)에 맞게 재생성합니다.
    """
    
    # set prompt for gpt model
    prompt = f"""
    I'll send you the article body. Please return the value in format like below.
    dict(“rewrite_article": string, “category": string, "keywords": list(str))

    1. Rewrite the article text to a {level} level and put it as the first part of the return value. You must finish article within 50 centences, and line it up appropriately to make it easier to read. Also, be original and must not duplicate any of the original sentences.
    2. Select the category of t]orts, business, innovation, culture, travel, or earth, and enter it as the second value in the return value.
    3. Select 5-7 important words from the article in purpose of learning English and put them as the 3rd part of the return value

    Article Text: {context}
    """
    
    # get response form gpt model
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that rewrites articles for teaching English."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=2048,
        temperature=0.5
    )
    
    # return response text
    response_dict = response.model_dump()
    response_message = response_dict["choices"][0]["message"]["content"]
    return response_message


In [30]:
leveled_text = convert_txt_to_steps(context, "elementary school")
results = eval(leveled_text)
results

{'rewrite_article': '\n    On New Year\'s Day, there was a terrible attack in New Orleans that resulted in the deaths of 15 people. \n    The police believe the person responsible did not act alone. \n    Shamsud-Din Jabbar, a 42-year-old man from the United States, drove a truck into a crowd on a busy street. \n    After hitting people, he got out of the truck and started shooting. \n    The police shot him at the scene. \n\n    Inside the truck, the FBI found a flag of a group called the Islamic State. \n    They also found two bombs nearby. \n    An FBI agent named Alethea Duncan said they think Jabbar was not the only one involved and are looking into this as an act of terrorism. \n\n    President Joe Biden mentioned that investigators are checking if this attack is connected to an explosion at a hotel in Las Vegas. \n    So far, they have not found any links to the Islamic State from that incident. \n    The police are also looking into whether the attack is related to Donald Trum

### Unsplash Test

In [2]:
import os
import requests
from dotenv import load_dotenv

load_dotenv(verbose=True)
unsplash_access_key = os.getenv("UNSPLASH_ACCESS_KEY")

url = f"https://api.unsplash.com/photos/random?client_id={unsplash_access_key}"
response = requests.get(url=url)
response.json

<bound method Response.json of <Response [200]>>

In [12]:
img_dict = response.json()
print(img_dict)
print(type(img_dict))

{'id': 'uTR1zs3Ky78', 'slug': 'a-cobblestone-street-lined-with-old-buildings-uTR1zs3Ky78', 'alternative_slugs': {'en': 'a-cobblestone-street-lined-with-old-buildings-uTR1zs3Ky78', 'es': 'una-calle-empedrada-bordeada-de-edificios-antiguos-uTR1zs3Ky78', 'ja': '古い建物が立ち並ぶ石畳の通り-uTR1zs3Ky78', 'fr': 'une-rue-pavee-bordee-de-vieux-batiments-uTR1zs3Ky78', 'it': 'una-strada-acciottolata-fiancheggiata-da-vecchi-edifici-uTR1zs3Ky78', 'ko': '오래된-건물이-늘어선-조약돌-거리-uTR1zs3Ky78', 'de': 'eine-kopfsteinpflasterstrasse-die-von-alten-gebauden-gesaumt-ist-uTR1zs3Ky78', 'pt': 'uma-rua-de-paralelepipedos-ladeada-por-edificios-antigos-uTR1zs3Ky78'}, 'created_at': '2024-12-18T10:04:44Z', 'updated_at': '2024-12-31T21:05:32Z', 'promoted_at': '2024-12-23T00:01:00Z', 'width': 8192, 'height': 5464, 'color': '#f3f3f3', 'blur_hash': 'LkMZ]*-;D%Vs_NxuWBWB?vbboftR', 'description': None, 'alt_description': 'A cobblestone street lined with old buildings', 'breadcrumbs': [], 'urls': {'raw': 'https://images.unsplash.com/photo

In [15]:
img_dict["urls"]["small"]

'https://images.unsplash.com/photo-1734515932217-a5f737525cc0?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w2OTIwNTB8MHwxfHJhbmRvbXx8fHx8fHx8fDE3MzU3MjM3MDF8&ixlib=rb-4.0.3&q=80&w=400'

In [18]:
url = f"https://api.unsplash.com/search/photos?client_id={unsplash_access_key}"
response = requests.get(url=url, params={"query": "black cat, blanket"})
img_dict = response.json()
print(img_dict["results"][0]["urls"])

{'raw': 'https://images.unsplash.com/photo-1512528368222-38761bbe68dd?ixid=M3w2OTIwNTB8MHwxfHNlYXJjaHwxfHxibGFjayUyMGNhdCUyQyUyMGJsYW5rZXR8ZW58MHx8fHwxNzM1NzI5MTQ2fDA&ixlib=rb-4.0.3', 'full': 'https://images.unsplash.com/photo-1512528368222-38761bbe68dd?crop=entropy&cs=srgb&fm=jpg&ixid=M3w2OTIwNTB8MHwxfHNlYXJjaHwxfHxibGFjayUyMGNhdCUyQyUyMGJsYW5rZXR8ZW58MHx8fHwxNzM1NzI5MTQ2fDA&ixlib=rb-4.0.3&q=85', 'regular': 'https://images.unsplash.com/photo-1512528368222-38761bbe68dd?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w2OTIwNTB8MHwxfHNlYXJjaHwxfHxibGFjayUyMGNhdCUyQyUyMGJsYW5rZXR8ZW58MHx8fHwxNzM1NzI5MTQ2fDA&ixlib=rb-4.0.3&q=80&w=1080', 'small': 'https://images.unsplash.com/photo-1512528368222-38761bbe68dd?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w2OTIwNTB8MHwxfHNlYXJjaHwxfHxibGFjayUyMGNhdCUyQyUyMGJsYW5rZXR8ZW58MHx8fHwxNzM1NzI5MTQ2fDA&ixlib=rb-4.0.3&q=80&w=400', 'thumb': 'https://images.unsplash.com/photo-1512528368222-38761bbe68dd?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w2OTIw

### Sentence, Word Sim Test

In [19]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 43.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 45.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 50.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 54.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 42.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 17.7 MB/s eta 0:00:00


In [29]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 모델 로드
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [32]:
sentences = results["rewrite_article"]
sentence_embeddings = model.encode(sentences)

In [33]:
word_embedding = []
for word in results["keywords"]:
    word_embedding.append(model.encode(word))

In [50]:
import numpy as np

word_sims = []
# 4. 코사인 유사도 계산
for em in word_embedding:
    data = np.array(em)
    print(f"origin shape: {data.shape}")
    reshaped_data = data.reshape(1, -1)
    print(f"reshaped shape: {reshaped_data.shape}")
    sim = cosine_similarity(reshaped_data, sentence_embeddings)
    # word_sims.append(sim)

# # 5. 결과 출력
# for i, sentence in enumerate(word_sims):
#     print(f"Similarity between '{word}' and 'sentence': {word_sims[0][i]:.4f}")

origin shape: (384,)
reshaped shape: (1, 384)


ValueError: Expected 2D array, got 1D array instead:
array=[-2.75533255e-02  6.37150183e-02 -2.52408516e-02 -8.55092052e-03
  7.04711676e-02 -2.82979794e-02  1.53779322e-02 -7.77567178e-02
  7.08258823e-02 -4.10191454e-02  7.30010420e-02  1.02148131e-02
  1.23317473e-01  2.69446149e-02  3.53877135e-02  3.31917778e-02
  2.52374019e-02 -4.03871909e-02 -8.33430588e-02  1.66621059e-02
 -5.44863157e-02  1.65747311e-02  2.40899492e-02 -1.16899703e-02
 -3.10931690e-02 -1.40323145e-02  2.61776429e-02  3.19667943e-02
 -1.11051418e-01  3.19443457e-02  1.21237926e-01 -3.43433139e-03
 -1.13666110e-01  3.55782476e-03 -9.41712235e-04 -2.89851557e-02
  5.18659316e-02  6.33020177e-02  7.52802044e-02  4.34009731e-02
  3.19648832e-02 -1.29360303e-01 -2.55632289e-02 -2.19880100e-02
  7.79529987e-03  6.70087943e-03 -3.76795791e-02  1.26106944e-03
  1.22850567e-01 -3.25954184e-02 -6.08828887e-02  4.76699769e-02
  4.93462645e-02 -4.42912243e-02  2.68398020e-02 -1.59823805e-01
  1.42589388e-02  9.12939943e-0